In [ ]:
# need to convert the data into embeddings first

In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from tqdm import tqdm

In [2]:
data = np.load('data/ner_trigger_dataset.npz')

In [3]:
print(f"Data keys: {data.keys()}")

Data keys: KeysView(NpzFile 'data/ner_trigger_dataset.npz' with keys: X, y)


In [5]:
X = data['X']
y = data['y']

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

print(X[0])
print(y[0])

X shape: (2148223, 6)
y shape: (2148223,)
['What' 'kind' 'of' 'memory' '?' 'We']
0


In [6]:
for i in range(20):
    print(X[i])
    print(y[i])

['What' 'kind' 'of' 'memory' '?' 'We']
0
['kind' 'of' 'memory' '?' 'We' 'respectfully']
0
['of' 'memory' '?' 'We' 'respectfully' 'invite']
0
['memory' '?' 'We' 'respectfully' 'invite' 'you']
0
['?' 'We' 'respectfully' 'invite' 'you' 'to']
0
['We' 'respectfully' 'invite' 'you' 'to' 'watch']
0
['respectfully' 'invite' 'you' 'to' 'watch' 'a']
0
['invite' 'you' 'to' 'watch' 'a' 'special']
0
['you' 'to' 'watch' 'a' 'special' 'edition']
0
['to' 'watch' 'a' 'special' 'edition' 'of']
0
['watch' 'a' 'special' 'edition' 'of' 'Across']
0
['a' 'special' 'edition' 'of' 'Across' 'China']
1
['special' 'edition' 'of' 'Across' 'China' '.']
1
['edition' 'of' 'Across' 'China' '.' 'WW']
0
['of' 'Across' 'China' '.' 'WW' 'II']
0
['Across' 'China' '.' 'WW' 'II' 'Landmarks']
0
['China' '.' 'WW' 'II' 'Landmarks' 'on']
0
['.' 'WW' 'II' 'Landmarks' 'on' 'the']
0
['WW' 'II' 'Landmarks' 'on' 'the' 'Great']
0
['II' 'Landmarks' 'on' 'the' 'Great' 'Earth']
0
